In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Carregando os dados
treino = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/train.csv')
teste = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/test.csv')

In [ ]:
treino.shape, teste.shape 

In [ ]:
# Juntando os dataframes
data = treino.append(teste)
data.shape

In [ ]:
data.head().T

In [ ]:
#Verificando os dados
data.info()

In [ ]:
# Remoção das colunas Unnamed e id
data.drop(['Unnamed: 0','id'],axis=1,inplace=True)

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
#Corrigir valores nulos
data.isna().sum()

In [ ]:
#Mudar valores nulos para 0
data.fillna(0, inplace = True)
data.isna().sum()

# Análise Exploratória de Dados

In [ ]:
#Importando bibliotecas
import matplotlib.pyplot as plt
import seaborn as sns 
from matplotlib import pyplot
import numpy as np

In [ ]:
data.describe()

In [ ]:
data['satisfaction'].value_counts()

In [ ]:
data['satisfaction'].value_counts().head().plot.barh(color = ['mediumorchid','thistle'])

Observa-se que o numero de pessoas "Neutras" ou Insatisfeitas é maior do que o número de pessoas Satisfeitas.

In [ ]:
data['Gender'].value_counts()

In [ ]:
data['Gender'].value_counts().head().plot.barh(color = ['mediumorchid','thistle'])

Há mais consumidores pertencentes ao gênero Feminino do que masculino, mas a diferença e é de apenas 1.918 consumidores.

In [ ]:
data['Customer Type'].value_counts()

In [ ]:
data['Customer Type'].describe()

In [ ]:
data['Customer Type'].value_counts().head().plot.barh(color = ['mediumorchid','thistle'])

A maior parte dos consumidores são de clientes leais, ou seja, fiéis a marca.

In [ ]:
data['Type of Travel'].value_counts()

In [ ]:
data['Type of Travel'].value_counts().head().plot.barh(color = ['mediumorchid','thistle'])

Em relação a motivação da viagem, há grande maioria foram Viagem de Negócios/ Viagens a trabalho (Business Travel).

In [ ]:
classe = data['Class'].value_counts()
classe

In [ ]:
classe = data['Class'].value_counts().head().plot.barh(color = ['purple', 'mediumorchid','thistle'])

Quanto a Classe de serviços dos voos, a maior parte das passagens vendidas no período foram da classe "Business" (Classe Executiva), a segunda maior são do tipo "Eco" (Econômica).

In [ ]:
correlation = data.corr()
correlation

In [ ]:
f, ax = plt.subplots(figsize=(15, 10))

mask = np.triu(np.ones_like(correlation,dtype=bool))
sns.heatmap(correlation,cmap="rocket_r",mask=mask,annot=True, linewidths=.5)

In [ ]:
sns.barplot(
data = data, 
x = "Flight Distance", 
y = "satisfaction",
palette = "rocket")

# Criação de Dummys

In [ ]:
# Transformar os valores da coluna target em numéricos
data['satisfaction'] = data['satisfaction'].replace({'satisfied': 1, 'neutral or dissatisfied' : 0}).astype(int)

data.select_dtypes('object').head() #quais colunas são do tipo 'object'

In [ ]:
# Criando variaveis Dummy: 
data2 = pd.get_dummies(data, columns=['Gender', 'Customer Type', 'Type of Travel','Class'])
data2.head()

In [ ]:
data2.shape

In [ ]:
data2.info()

# Treinamento do modelo

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Separando treino e teste
train, test = train_test_split(data2, test_size=0.20, random_state=42)

In [ ]:
# Separando treino e validação
train, valid = train_test_split(train, test_size=0.20, random_state=42)

In [ ]:
train.shape, valid.shape, test.shape

In [ ]:
#coluna de entrada
feats = [c for c in data2.columns if c not in ['satisfaction']]
feats

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 200, random_state = 42)

In [ ]:
# treinar o modelo
rf.fit(train[feats], train['satisfaction'])

In [ ]:
# Prevendo os dados de validação
pred = rf.predict(valid[feats])
pred

# Avaliação do Modelo

In [ ]:
from sklearn.metrics import accuracy_score #importando métrica

accuracy_score(valid['satisfaction'], pred) #Acurácia das previsões de validação

In [ ]:
# Medindo a acurácia nos dados de teste
pred_teste = rf.predict(test[feats])

accuracy_score(test['satisfaction'], pred_teste)

In [ ]:
data['satisfaction'].value_counts(normalize = True) #Coluna satisfaction do Dataframe completo

In [ ]:
data2.info()

In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada)
pd.Series(rf.feature_importances_, index = feats).sort_values().plot.barh()

In [ ]:
# importando a bilbioteca para plotar o gráfico de Matriz de Confusão
import scikitplot as skplt

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(valid['satisfaction'], pred)